#  Foodie Score
Guy Shavit 
Written: 3/1/2021 
 
## Introduction
The target market is people that place high importance on restaurant options and are looking for places to move.  
This specific project will use as an example people that are moving from the Williamsburg neighborhood in Brooklyn, NY to Miami but it can be generalized in the future for any source and destination.
The project will calculate a "foodie" score for each neighborhood in Miami which will incroporate the amount of restaurants within walking distance and their diversity.  
The project will also rank the neighboroods in the target location (Miami) based on how similar they are to the source neighborhood(Williamsburg, Brooklyn) inand restauant offerings.  



## Data
Location Data:
1. The coordinates of Williamsburg are avialable in the newyork_data.json
2. The names and coordinates of the Miami neighborhoods will be scraped from Wikipedia

For the Foodie Score of a location we will need the following data:
1. Number of restaurants within 1000m of each neighborhood coordinate
2. Types of cuisine of each restaurant
3. Rating of each restaurant

For the simalarity ranking to Williamsburg, we will create a frequency dataframe of restaurants in each neighborhood in Miami and compare it to that of Williamsburg

## Methodology

### Download Dependancies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library
import branca

from math import sqrt


print('Libraries imported.')

Libraries imported.


### Prepare location data

Get the newyork_data.json for Williamsburg coordinates

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
Williamsburg = pd.DataFrame()
Williamsburg['Neighborhood'] = ['Williamsburg']
   
neighborhoods_data = newyork_data['features']
for data in neighborhoods_data:
    if data['properties']['name'] == 'Williamsburg':
        Williamsburg_latlon = data['geometry']['coordinates']
        Williamsburg['Latitude'] = Williamsburg_latlon[1]
        Williamsburg['Longitude'] = Williamsburg_latlon[0]
Williamsburg.head()


Data downloaded!


,Neighborhood,Latitude,Longitude
0,Williamsburg,40.707144,-73.958115


Get Miami neighborhood information 

In [3]:
!wget -O List_of_neighborhoods_in_Miami https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami
df = pd.read_html('List_of_neighborhoods_in_Miami')

--2021-03-12 00:44:12--  https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60849 (59K) [text/html]
Saving to: ‘List_of_neighborhoods_in_Miami’

List_of_neighborhoo 100%[===================>]  59.42K  --.-KB/s    in 0.07s   

2021-03-12 00:44:13 (891 KB/s) - ‘List_of_neighborhoods_in_Miami’ saved [60849/60849]



Clean up Miami dataframe:
1. drop unnecessary columns
2. drop neigborhoods with missing data
3. replace the coordinates text with a Latitude and Longitude numeric values

In [4]:
Miami_Neighborhood = df[0]
Miami_Neighborhood.drop(columns=['Demonym','Sub-neighborhoods','Population2010','Population/Km²'], inplace=True)
Miami_Neighborhood.dropna(inplace=True)
Miami_Neighborhood.reset_index(drop=True, inplace=True)
Miami_Neighborhood[['Latitude','Longitude']] = Miami_Neighborhood.Coordinates.str.split("-",expand=True)
Miami_Neighborhood[['Latitude','Longitude']] = Miami_Neighborhood[['Latitude','Longitude']].apply(pd.to_numeric) 
Miami_Neighborhood['Longitude'] = -Miami_Neighborhood['Longitude']
Miami_Neighborhood.drop(columns=['Coordinates'], inplace=True)
Miami_Neighborhood.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257


Creating a Neighborhoods dataframe that has all the Miami neighborhoods and Williamsburg 

In [5]:
Neighborhoods = Miami_Neighborhood.append(Williamsburg, ignore_index=True)

Get location of Miami to initialize map

In [6]:
address = 'Miami, Florida'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
Miami_latitude = location.latitude
Miami_longitude = location.longitude
print('The geograpical coordinate of Miami are {}, {}.'.format(Miami_latitude, Miami_longitude))

The geograpical coordinate of Miami are 25.7741728, -80.19362.


Create map with Miami Neighboorhoods

In [7]:
map_Miami = folium.Map(location=[Miami_latitude, Miami_longitude], zoom_start=12)

#add markers to map
for lat, lng, Neighborhood in zip(Miami_Neighborhood['Latitude'], Miami_Neighborhood['Longitude'], Miami_Neighborhood['Neighborhood']):
    if Miami_Neighborhood['Neighborhood'] is not "Williamsburg":
        label = '{}'.format(Neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [float(lat), float(lng)],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Miami)  
    
map_Miami

### Get Restaurant Data

Initialize Foursquare API login data

In [8]:
CLIENT_ID = 'ULAMQLZ3O2MJYTHU2QTK0QYRUZ3YP3ZNGOSFZ45PG2R4TQ0I' 
CLIENT_SECRET = 'Z5LBQMM1WEPQSJJWIQ3ALKHCVFLD4VR1DTSX1MCE0ZURWW3O' 
ACCESS_TOKEN = 'MZI1APQNRECXU5PRJZMY4BLZUHRGYZEZ24SU3NEYXMKWGV03' 

VERSION = '20200212' # Foursquare API version
LIMIT = 100

Function that receives the name of the Neighborood and location and adds to dataframe all the restaurants within 1000m of that location

In [9]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section=food&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create the Miami_venues dataframe with restaurant data for all Miami Neighborhoods

In [11]:
Venues = getNearbyVenues(names=Neighborhoods['Neighborhood'],
                                   latitudes=Neighborhoods['Latitude'],
                                   longitudes=Neighborhoods['Longitude']
                                  )

In [12]:
print(Venues.shape)
Venues.head()

(1140, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allapattah,25.815,-80.224,Little Caesars Pizza,25.809315,-80.224240,Pizza Place
1,Allapattah,25.815,-80.224,Checkers,25.809220,-80.225682,Fast Food Restaurant
2,Allapattah,25.815,-80.224,Tipico Sur-Americano,25.809427,-80.223450,South American Restaurant
3,Allapattah,25.815,-80.224,Cafeteria Amarilis Dominican Food,25.808877,-80.223655,Spanish Restaurant
4,Allapattah,25.815,-80.224,amarillis,25.808877,-80.223655,Latin American Restaurant


### Create Foodie score for Miami Neighborhoods

For each Neighborhood in Miami, count how many total restaurants there are and how many different types of restaurant cuisine.
Foodie score is 1 point for each restaurant, 2 points for each different type and then normalized for score between 0 and 100

In [13]:
score = []

for index, Neighborhood in Miami_Neighborhood.iterrows():
        NumRest = Venues.loc[Venues['Neighborhood'] == Neighborhood['Neighborhood']].shape[0]
        UniqRest = Venues.loc[Venues['Neighborhood'] == Neighborhood['Neighborhood']].nunique()['Venue Category']
        score.append(NumRest + 2*UniqRest)
norm_score = [round((float(i)/max(score))*100,2) for i in score]
Miami_Neighborhood['Score'] = norm_score
Miami_Neighborhood

,Neighborhood,Latitude,Longitude,Score
0,Allapattah,25.815,-80.224,14.04
1,Arts & Entertainment District,25.799,-80.190,95.51
2,Brickell,25.758,-80.193,70.22
3,Buena Vista,25.813,-80.192,83.71
4,Coconut Grove,25.712,-80.257,5.62
5,Coral Way,25.750,-80.283,11.24
6,Design District,25.813,-80.193,78.65
7,Downtown,25.774,-80.193,100.00
8,Edgewater,25.802,-80.190,96.63
9,Flagami,25.762,-80.316,25.84


In order to create choropleth map of Miami by Foodie score we download a geojson file.

In [14]:
# download countries geojson file
!wget --quiet https://raw.githubusercontent.com/blackmad/neighborhoods/master/miami.geojson
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


In [15]:
Miami_geo = r'miami.geojson' # geojson file
#Miami_geo = r'2f54a0cbd67046f2bd100fb735176e6c_0.geojson' # geojson file


Miami_map = folium.Map(location=[Miami_latitude, Miami_longitude], zoom_start=12)

Miami_map.choropleth(
    geo_data=Miami_geo,
    data=Miami_Neighborhood,
    columns=['Neighborhood', 'Score'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Foodie Score'
)


# display map
Miami_map

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/folium/folium.py:413: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


That is helpful, but unfortunatly not all neighborhoods in our dataset are in the geojson file so lets create a new map with color coded markers for all neighborhoods

In [16]:
# set color scheme for the markers

map_Miami = folium.Map(location=[Miami_latitude, Miami_longitude], zoom_start=12)
colormap = branca.colormap.linear.YlOrRd_09.scale(0, 100)
colormap = colormap.to_step(index=[0, 25, 50, 75, 100])
colormap.caption = 'Foodie Score'
colormap.add_to(map_Miami)

for lat, lng, Neighborhood, Score in zip(Miami_Neighborhood['Latitude'], Miami_Neighborhood['Longitude'], Miami_Neighborhood['Neighborhood'],Miami_Neighborhood['Score']):
    label = folium.IFrame("{} <br> Foodie Score: {}".format(Neighborhood,Score),width=200,height=60)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=15,
        popup=label,
        color=colormap(Score),
        opacity=0.7,
        fill=True,
        fill_color=colormap(Score),
        fill_opacity=0.7,
        parse_html=False).add_to(map_Miami)  



    
map_Miami

## Compare Miami neighborhoods to Williamsburg in restaurant profile

Create a table with the frequency of each restaurant cuisine in each neighborhood

In [17]:
# one hot encoding
Miami_onehot = pd.get_dummies(Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
Miami_onehot['Neighborhood'] = Venues['Neighborhood'] 

# move neighbourhood column to the first column
fixed_columns = [Miami_onehot.columns[-1]] + list(Miami_onehot.columns[:-1])
Miami_onehot = Miami_onehot[fixed_columns]

Miami_grouped = Miami_onehot.groupby('Neighborhood').mean().reset_index()
Miami_grouped


,Neighborhood,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brasserie,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Peruvian Restaurant,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Allapattah,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.0,0.222222,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
1,Arts & Entertainment District,0.000000,0.030000,0.000000,0.020000,0.010000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.070000,0.000000,0.010000,0.020000,0.000000,0.000000,0.00,0.020000,0.010000,0.010000,0.010000,0.00,0.000000,0.010000,0.010000,0.0,0.000000,0.000000,0.000000,0.130000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.050000,0.030000,0.00,0.00,0.020000,0.010000,0.040000,0.000000,0.020000,0.020000,0.070000,0.010000,0.01,0.140000,0.000000,0.050000,0.00,0.030000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.000000
2,Brickell,0.014925,0.059701,0.000000,0.044776,0.029851,0.000000,0.014925,0.029851,0.000000,0.000000,0.000000,0.000000,0.029851,0.000000,0.000000,0.059701,0.000000,0.000000,0.014925,0.000000,0.000000,0.00,0.014925,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.014925,0.0,0.014925,0.000000,0.000000,0.000000,0.029851,0.000000,0.014925,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.104478,0.044776,0.00,0.00,0.044776,0.000000,0.044776,0.044776,0.000000,0.014925,0.104478,0.000000,0.00,0.044776,0.000000,0.014925,0.00,0.044776,0.000000,0.000000,0.014925,0.029851,0.029851,0.014925,0.000000,0.014925,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
3,Buena Vista,0.000000,0.028169,0.014085,0.014085,0.014085,0.000000,0.000000,0.042254,0.014085,0.000000,0.014085,0.014085,0.014085,0.000000,0.000000,0.070423,0.000000,0.014085,0.014085,0.000000,0.000000,0.00,0.014085,0.042254,0.000000,0.000000,0.00,0.000000,0.014085,0.014085,0.0,0.014085,0.014085,0.000000,0.014085,0.000000,0.000000,0.014085,0.000000,0.00,0.014085,0.014085,0.014085,0.00,0.070423,0.042254,0.00,0.00,0.028169,0.014085,0.028169,0.000000,0.000000,0.014085,0.140845,0.000000,0.00,0.042254,0.000000,0.042254,0.00,0.042254,0.014085,0.000000,0.014085,0.000000,0.014085,0.000000,0.014085,0.000000,0.00,0.00,0.00,0.014085,0.000000,0.014085,0.000000
4,Coconut Grove,0.00000

Create a dataframe of the 10 most common restaurants in each neighborhood in Miami

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Miami_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Miami_neighborhoods_venues_sorted['Neighborhood'] = Miami_grouped['Neighborhood']

for ind in np.arange(Miami_grouped.shape[0]):
    Miami_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Miami_grouped.iloc[ind, :], num_top_venues)

Miami_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allapattah,Food,Food Truck,Fast Food Restaurant,Bakery,Latin American Restaurant,Pizza Place,South American Restaurant,Spanish Restaurant,Diner,Comfort Food Restaurant
1,Arts & Entertainment District,Restaurant,Food Truck,Pizza Place,Café,Italian Restaurant,Sandwich Place,Mexican Restaurant,Seafood Restaurant,American Restaurant,Japanese Restaurant
2,Brickell,Pizza Place,Italian Restaurant,American Restaurant,Café,Argentinian Restaurant,Middle Eastern Restaurant,Restaurant,Mexican Restaurant,Latin American Restaurant,Seafood Restaurant
3,Buena Vista,Pizza Place,Italian Restaurant,Café,Restaurant,Sandwich Place,Seafood Restaurant,Japanese Restaurant,Bakery,Deli / Bodega,Latin American Restaurant
4,Coconut Grove,Fried Chicken Joint,American Restaurant,Café,Eastern European Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Dumpling Restaurant


Lets look at Williamsburg

In [19]:
Miami_neighborhoods_venues_sorted[Miami_neighborhoods_venues_sorted["Neighborhood"]=='Williamsburg']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Williamsburg,American Restaurant,Pizza Place,Italian Restaurant,Bakery,Mexican Restaurant,Chinese Restaurant,Café,Mediterranean Restaurant,Sushi Restaurant,Burger Joint


Let's calculate the Euclidean distance between the each of the Miami neighborhoods and Williamsburg

In [20]:
similarity = []
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)):
        distance += ((row1[i]) - (row2[i]))**2
    return sqrt(distance)

     
Williamsburg_grouped_clustering = Miami_grouped[Miami_grouped['Neighborhood']=='Williamsburg'].drop('Neighborhood', 1).values.tolist()
Miami_grouped_clustering = Miami_grouped[Miami_grouped['Neighborhood']!='Williamsburg'].drop('Neighborhood', 1).values.tolist()

for i in range(len(Miami_grouped_clustering)):
    distance = euclidean_distance(Williamsburg_grouped_clustering[0], Miami_grouped_clustering[i])
    similarity.append(distance)
# turn the Euclidean distance into a score between 0 and 100 where 100 is exactly the same as Williamsburg
similarity_score = [round((1-float(i))*100,2) for i in similarity]
Miami_Neighborhood['Similarity'] = similarity_score
Miami_Neighborhood

,Neighborhood,Latitude,Longitude,Score,Similarity
0,Allapattah,25.815,-80.224,14.04,64.15
1,Arts & Entertainment District,25.799,-80.190,95.51,79.17
2,Brickell,25.758,-80.193,70.22,85.70
3,Buena Vista,25.813,-80.192,83.71,85.84
4,Coconut Grove,25.712,-80.257,5.62,41.93
5,Coral Way,25.750,-80.283,11.24,64.69
6,Design District,25.813,-80.193,78.65,85.87
7,Downtown,25.774,-80.193,100.00,82.45
8,Edgewater,25.802,-80.190,96.63,82.22
9,Flagami,25.762,-80.316,25.84,68.50


Lets put these similarity scores on the map

In [21]:
# set color scheme for the markers

map_Miami = folium.Map(location=[Miami_latitude, Miami_longitude], zoom_start=12)
colormap = branca.colormap.linear.YlOrRd_09.scale(0, 100)
colormap = colormap.to_step(index=[0, 10, 20, 30, 40, 50, 60, 70, 80 ,90 ,100])
colormap.caption = 'Williamsburg Similarity Score'
colormap.add_to(map_Miami)

for lat, lng, Neighborhood, Similarity in zip(Miami_Neighborhood['Latitude'], Miami_Neighborhood['Longitude'], Miami_Neighborhood['Neighborhood'],Miami_Neighborhood['Similarity']):
    label = folium.IFrame("{} <br> Williamsburg Similarity Score: {}".format(Neighborhood,Similarity),width=200,height=60)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=15,
        popup=label,
        color=colormap(Similarity),
        opacity=0.7,
        fill=True,
        fill_color=colormap(Similarity),
        fill_opacity=0.7,
        parse_html=False).add_to(map_Miami)  



    
map_Miami

## Results and Discussion
The 5 neighborhoods most similiar to Williamsburg are:

In [22]:
Miami_Neighborhood.sort_values(by=['Similarity'], ascending=False).head()

,Neighborhood,Latitude,Longitude,Score,Similarity
6,Design District,25.813,-80.193,78.65,85.87
3,Buena Vista,25.813,-80.192,83.71,85.84
2,Brickell,25.758,-80.193,70.22,85.70
15,Midtown,25.807,-80.193,100.00,84.38
7,Downtown,25.774,-80.193,100.00,82.45


The 5 neighboorhoods with the highest "Foodie Score" are:

In [23]:
Miami_Neighborhood.sort_values(by=['Score'], ascending=False).head()

,Neighborhood,Latitude,Longitude,Score,Similarity
15,Midtown,25.807,-80.193,100.00,84.38
7,Downtown,25.774,-80.193,100.00,82.45
23,Wynwood,25.804,-80.199,97.75,80.92
8,Edgewater,25.802,-80.190,96.63,82.22
1,Arts & Entertainment District,25.799,-80.190,95.51,79.17


Based on this information, Midtown and Downtown appear to be good places to look.
A Choropleth is very useful but it's important to verify that the sources include the same data.

## Conclusion
The analysis above is done for one source neighborhood and one destination city but could be generalized for any source and destination and even could look for destination neighboorhoods in multiple cities.
The most important limitation factor is a reliable source of neighborhood information for all cities.